In [ ]:
import cv2
import face_recognition
import pytesseract
import numpy as np
import os
from threading import Thread, Lock
from queue import Queue
import time
import platform
import re

# ========== CONFIGURATION ==========
CONFIG = {
    "FACE_MATCH_THRESHOLD": 0.6,
    "PLATE_OCR_CONFIG": r'--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789',
    "MIN_PLATE_CHARS": 3,
    "FRAME_SKIP": 3,  # Process plates every N frames
    "CAMERAS": [
        {
            "name": "Mobile",
            "src": "http://192.168.29.231:8080/video",
            "role": "plates",
            "width": 1280,  # Increased resolution for better plate detection
            "height": 720,
            "fps": 15
        },
        {
            "name": "Laptop",
            "src": 0,
            "role": "faces",
            "width": 640,  # Increased resolution for better face detection
            "height": 480,
            "fps": 20
        }
    ]
}

# ========== INITIALIZATION ==========
print("🔍 Loading reference faces...")
try:
    known_face_encodings = []
    known_face_names = []
    
    reference_folder = "reference_images"
    os.makedirs(reference_folder, exist_ok=True)
    reference_image_path = os.path.join(reference_folder, "person.jpg")
    
    if not os.path.exists(reference_image_path):
        raise FileNotFoundError(f"Reference image not found at {reference_image_path}")
    
    ref_image = face_recognition.load_image_file(reference_image_path)
    ref_encodings = face_recognition.face_encodings(ref_image)
    
    if not ref_encodings:
        raise ValueError("No faces found in reference image")
    
    known_face_encodings.append(ref_encodings[0])
    known_face_names.append("Authorized Person")
    print(f"✅ {len(known_face_encodings)} reference faces loaded")
except Exception as e:
    print(f"❌ Face load error: {e}")
    exit()

# Tesseract setup
if platform.system() == 'Windows':
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# ========== OPTIMIZED VIDEO STREAM ==========
class VideoStream:
    def __init__(self, src, name="Camera", width=640, height=480, fps=30):
        self.stream = cv2.VideoCapture(src)
        if not self.stream.isOpened():
            raise RuntimeError(f"Cannot open {name} camera")
        
        self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        self.stream.set(cv2.CAP_PROP_FPS, fps)
        self.stream.set(cv2.CAP_PROP_BUFFERSIZE, 2)  # Increased buffer size slightly
        self.name = name
        self.role = "faces" if name == "Laptop" else "plates"
        self.frame_queue = Queue(maxsize=2)
        self.lock = Lock()
        self.running = False
        self.width = width
        self.height = height
        
    def start(self):
        self.running = True
        self.thread = Thread(target=self.update, daemon=True)
        self.thread.start()
        return self
    
    def update(self):
        while self.running:
            ret, frame = self.stream.read()
            if ret:
                # Check if frame matches the requested dimensions, if not, resize
                if frame.shape[1] != self.width or frame.shape[0] != self.height:
                    frame = cv2.resize(frame, (self.width, self.height))
                
                with self.lock:
                    if self.frame_queue.full():
                        self.frame_queue.get()
                    self.frame_queue.put(frame)
            time.sleep(0.01)  # More frequent updates
    
    def read(self):
        with self.lock:
            return self.frame_queue.get() if not self.frame_queue.empty() else None
        
    def stop(self):
        self.running = False
        if hasattr(self, 'thread'):
            self.thread.join()
        self.stream.release()

# ========== PROCESSING FUNCTIONS ==========
def detect_faces(frame):
    if frame is None:
        return None
    
    try:
        # Convert to RGB (face_recognition uses RGB)
        rgb_frame = frame[:, :, ::-1]
        
        # Scale down for faster processing if needed
        scale = 0.5
        small_frame = cv2.resize(rgb_frame, (0, 0), fx=scale, fy=scale)
        
        # Find face locations using HOG (faster than CNN)
        face_locations = face_recognition.face_locations(small_frame, model="hog")
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        
        # Draw debug info
        cv2.putText(frame, f"Faces found: {len(face_locations)}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Scale back up face locations
            top = int(top / scale)
            right = int(right / scale)
            bottom = int(bottom / scale)
            left = int(left / scale)
            
            matches = face_recognition.compare_faces(
                known_face_encodings, 
                face_encoding, 
                tolerance=CONFIG["FACE_MATCH_THRESHOLD"]
            )
            
            if True in matches:
                # Known face - green box
                color = (0, 0, 255)
                name = "Got Em"
            else:
                # Unknown face - red box
                color = (80, 80, 80)
                name = "Unidentified"
            
            # Draw bounding box and label
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    except Exception as e:
        print(f"⚠️ Face detection error: {e}")
        return frame

def detect_plates(frame):
    if frame is None:
        return None
    
    try:
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        gray = clahe.apply(gray)
        
        # Apply Gaussian blur to reduce noise
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        
        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY_INV, 11, 2)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Debug info
        cv2.putText(frame, f"Contours: {len(contours)}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        # Filter contours by area
        contours = [c for c in contours if cv2.contourArea(c) > 1000]
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]  # Top 5 candidates
        
        for contour in contours:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
            
            # More relaxed shape condition - 4-sided or more
            if 4 <= len(approx) <= 6:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                
                # License plate aspect ratio check
                if 2.0 < aspect_ratio < 6.0:
                    plate_img = gray[y:y+h, x:x+w]
                    
                    # Display potential license plate region
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
                    
                    # Check plate size before OCR
                    if plate_img.shape[0] > 0 and plate_img.shape[1] > 0:
                        # Apply additional preprocessing to the plate region
                        plate_img = cv2.resize(plate_img, (0, 0), fx=2, fy=2)  # Upscale
                        plate_img = cv2.GaussianBlur(plate_img, (3, 3), 0)
                        _, plate_img = cv2.threshold(plate_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
                        
                        # OCR with Tesseract
                        text = pytesseract.image_to_string(
                            plate_img,
                            config=CONFIG["PLATE_OCR_CONFIG"]
                        )
                        clean_text = re.sub(r'[^A-Z0-9]', '', text.upper())
                        
                        if len(clean_text) >= CONFIG["MIN_PLATE_CHARS"]:
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                            cv2.putText(frame, f"PLATE: {clean_text}", 
                                       (x, y-15), cv2.FONT_HERSHEY_SIMPLEX, 
                                       0.7, (255, 0, 0), 2)
        
        return frame
    except Exception as e:
        print(f"⚠️ Plate detection error: {e}")
        return frame

# ========== MAIN EXECUTION ==========
def main():
    print("📷 Starting camera streams...")
    cameras = []
    for cam in CONFIG["CAMERAS"]:
        try:
            camera = VideoStream(
                cam["src"],
                cam["name"],
                cam["width"],
                cam["height"],
                cam.get("fps", 15)
            ).start()
            cameras.append(camera)
            print(f"  ✅ {cam['name']} camera initialized for {cam['role']} - {cam['width']}x{cam['height']}")
        except Exception as e:
            print(f"  ❌ Failed to initialize {cam['name']} camera: {e}")
            cameras.append(None)
    
    try:
        # Create windows
        for cam in CONFIG["CAMERAS"]:
            cv2.namedWindow(cam["name"] + " Feed", cv2.WINDOW_NORMAL)
            if cam["name"] == "Laptop":
                cv2.resizeWindow(cam["name"] + " Feed", 640, 480)
            else:
                cv2.resizeWindow(cam["name"] + " Feed", 800, 600)
        
        print("🚀 Starting surveillance system (Press Q to quit)...")
        frame_counter = 0
        
        while True:
            start_time = time.time()
            
            for camera in cameras:
                if camera is None:
                    continue
                
                frame = camera.read()
                if frame is None:
                    continue
                
                # Role-based processing
                if camera.role == "faces":
                    processed_frame = detect_faces(frame)
                else:
                    if frame_counter % CONFIG["FRAME_SKIP"] == 0:
                        processed_frame = detect_plates(frame)
                    else:
                        processed_frame = frame
                
                if processed_frame is not None:
                    # Add camera resolution info
                    cv2.putText(processed_frame, 
                               f"{processed_frame.shape[1]}x{processed_frame.shape[0]}", 
                               (10, processed_frame.shape[0] - 10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    
                    cv2.imshow(camera.name + " Feed", processed_frame)
            
            frame_counter += 1
            
            # Exit on Q key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            # Maintain target FPS
            elapsed = time.time() - start_time
            target_delay = 1/25  # ~25 FPS combined (relaxed from 30 to accommodate higher resolution)
            if elapsed < target_delay:
                time.sleep(target_delay - elapsed)
    
    except KeyboardInterrupt:
        print("\n🛑 Received interrupt signal")
    except Exception as e:
        print(f"🔥 Critical error: {e}")
    finally:
        print("🛑 Stopping system...")
        for camera in cameras:
            if camera is not None:
                camera.stop()
        cv2.destroyAllWindows()
        print("✅ System stopped cleanly")

if __name__ == "__main__":
    main()